In [ ]:
import os
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import time
import os
from dotenv import load_dotenv

In [30]:
## Función

def apendizador_leads(ruta,sector):
    dfs = []

    for entry in os.scandir(ruta):
        if entry.is_file() and sector in entry.name:
            df = pd.read_csv(entry.path)

            # -------------------------
            # LIMPIEZA DE DATOS (ejemplo)
            # -------------------------
            df = df.dropna(subset=['Email'])          # elimina registros sin email
        

            # Guarda el dataframe limpio
            dfs.append(df)

    # Unir todos los dataframes en uno solo
    df_final = pd.concat(dfs, ignore_index=True)
    df_final.to_excel(ruta + f"/leads_{sector}_completo.xlsx", index=False)

    return df_final

In [31]:
df = apendizador_leads(r"C:\Users\ejgar\OneDrive\Documentos\GitHub\Leads_sheacher\data","Agencia_de_Viajes")

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Nombre     273 non-null    object
 1   Direccion  273 non-null    object
 2   Telefono   269 non-null    object
 3   Web        273 non-null    object
 4   Email      273 non-null    object
 5   Sector     273 non-null    object
 6   Ubicacion  273 non-null    object
dtypes: object(7)
memory usage: 15.1+ KB


In [27]:
df.head()

,Nombre,Direccion,Telefono,Web,Email,Sector,Ubicacion
0,Spencer Trip,"C. de Jacinto Benavente, 7, 2º B, 28100 Alcobe...",+34 633 82 20 14,http://www.spencertrip.com/,analia@spencertrip.com,Agencia de Viajes,Alcobendas
1,Halcón Viajes,"C. de Mariano Sebastian Izuel, 3, 28100 Alcobe...",+34 910 29 39 85,https://oficina.halconviajes.com/2748,alcobendas-izuel@halconviajes.com,Agencia de Viajes,Alcobendas
2,Viajamor.com,"Av. Matapiñonera, 11, Edif. 4, 106 Es en los E...",+34 916 51 21 75,https://www.viajamor.com/,info@viajamor.com,Agencia de Viajes,Alcobendas
3,Viajes Carrefour Alcobendas,"Carr. de Madrid a Burgos, Km. 14, 500, 28108 A...",+34 916 63 09 00,https://agencias.viajes.carrefour.es/viajes-ca...,es.viajes.alcobendas@carrefour.com,Agencia de Viajes,Alcobendas
4,Nautalia,"C. del Marqués de la Valdavia, 119, 28100 Alco...",+34 910 02 44 42,https://agencias.nautaliaviajes.com/alcobendas...,0645@nautaliaviajes.es,Agencia de Viajes,Alcobendas


In [41]:
# credenciales Gmail
load_dotenv()

EMAIL_REMITENTE = os.getenv("EMAIL_REMITENTE")
PASSWORD_APP = os.getenv("PASSWORD_APP")
SMTP_SERVER = os.getenv("SMTP_SERVER")
SMTP_PORT = int(os.getenv("SMTP_PORT"))

EMAIL_REMITENTE = EMAIL_REMITENTE
PASSWORD_APP = PASSWORD_APP

# servidor SMTP de Gmail
SMTP_SERVER = SMTP_SERVER
SMTP_PORT = SMTP_PORT

# conexión
server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
server.starttls()
server.login(EMAIL_REMITENTE, PASSWORD_APP)

for _, row in df.iterrows():
    # destinatario = row['email']
    destinatario = "muzquiz.10@gmail.com"  #Para pruebas
    # nombre = row['nombre']
    nombre = "Enrique" #Para pruebas
    mensaje = MIMEMultipart()
    mensaje['From'] = EMAIL_REMITENTE
    mensaje['To'] = destinatario
    mensaje['Subject'] = "Asunto del email"

    cuerpo = f"""
Hola {nombre},

Soy Enrique García, consultor freelance y fundador de EGM Analytics.

Ayudo a pequeños negocios y profesionales en dos áreas clave:

Business Intelligence y análisis de datos: organización y modelado de datos, automatización de informes, análisis para la toma de decisiones y mejora de procesos basados en datos.

Desarrollo web profesional: creación de páginas web claras y bien estructuradas que ayuden a tener presencia online, transmitir credibilidad y mostrar el negocio de forma profesional.

En muchos casos me encuentro con empresas que tienen datos dispersos o infrautilizados y, al mismo tiempo, una presencia digital que no refleja realmente lo que hacen o el valor que aportan.

Mi enfoque es práctico y cercano: soluciones sencillas, bien pensadas y adaptadas a cada negocio, sin complicaciones innecesarias.

Si te parece, podemos agendar una llamada breve (15–20 minutos, sin compromiso) para comentar tu situación y ver si puedo ayudarte en alguno de estos ámbitos.

Un saludo,


Enrique García
Consultor en Data, BI y Desarrollo Web
EGM Analytics
https://egmanalytics.com
"""
    mensaje.attach(MIMEText(cuerpo, 'plain'))

    try:
        server.send_message(mensaje)
        print(f"✔ Email enviado a {destinatario}")
        time.sleep(5)
    except Exception as e:
        print(f"❌ Error enviando a {destinatario}: {e}")
        time.sleep(5)

server.quit()


✔ Email enviado a muzquiz.10@gmail.com
✔ Email enviado a muzquiz.10@gmail.com


KeyboardInterrupt: 